## Module to scrap data from API based on Gspread provided

In [1]:
'''Getting the input values from Google Spread API'''

import gspread
import pandas as pd

def gspreadData():
    sheet_id='1jYrsjlx4xD8MSkFGM3mItJT59ytoc5_LZJgE6wMSWK0'
    sheet_name='CovidData'
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    url  
    df=pd.read_csv(url,error_bad_lines=False)
    print(df)
    return df



In [2]:
'''Scrapping value from CoWInAPI'''

import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as bs
import datetime
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

##this above import is to minismise SSL Security warning in Macbook


'''
Scripts to get the Vaccination available slot for covid-19 centers with ease 

Open public API: https://apisetu.gov.in/public/marketplace/api/cowin
'''

base = datetime.datetime.today()
print(base)

numdays = 25
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
print(date_list)
date_str = [x.strftime("%d-%m-%Y") for x in date_list]
date_str
age=58
print_flag = 'y'
print_flag = 'y'

headers={'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}

def vaccine_state_data():
    '''function to print the state and its correponsding district names'''

    try:
        ## Checking statecode avaialble
        response_state = rq.get("https://cdn-api.co-vin.in/api/v2/admin/location/states",headers=headers,verify=False)
        json_data_state = json.loads(response_state.text)
        print()
        json_data_state['states']

        data_dist=[]
        for state_id in range(1,40):
            ##Checking till 40 state code as i checked there are 40 statecode availble!
#             try:
#                 print("For State_code:{} State_name: {} ".format(state_id,next(i['state_name'] for i in json_data_state['states'] if i['state_id']==state_id)))
#             except:
#                 print("For State_code:{} State_name: {}".format(state_id,'Not Available') )
#                 pass
            response_dist = rq.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_id),headers=headers,verify=False)
            json_data = json.loads(response_dist.text)
            for _ in json_data["districts"]:
                
                # print(_["district_id"],'\t', _["district_name"])
                try:
                    data_dist.append([_["district_id"], _["district_name"],next(i['state_name'] for i in json_data_state['states'] if i['state_id']==state_id),state_id])
                except:
                    data_dist.append([_["district_id"], _["district_name"],None,state_id])
            # print("\n")

        df = pd.DataFrame(data_dist, columns = ['district_id', 'district_name','State_name','State_id'],index=None)
        df=df.sort_values(by=['State_id'], ascending=True)
        df_vacc_state = df.reindex(columns=['State_id','State_name','district_name','district_id'])
        print(len(df_vacc_state['district_id']))
        df_vacc_state['State_id'].nunique()
        return df_vacc_state


    except Exception as e:
        print("Exception ",e)



def vaccineSlotsByDist(age,district_name,state):
    try:
        available_df=[]
        age=age
        
        df_vacc_state=vaccine_state_data()
        try:
            print(df_vacc_state)
            DIST_ID=list(df_vacc_state[(df_vacc_state['State_name']== state) & (df_vacc_state['district_name']==district_name)]['district_id'])[0]

        except:
            print("No Combintaiont found")
            return None
            pass

        for inp_date in date_str:
            URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(DIST_ID, inp_date)
            response = rq.get(URL,headers=headers,verify=False)
            if response.ok:
                resp_json = response.json()
                # print(json.dumps(resp_json, indent = 1))
                if resp_json["centers"]:
                    print("Available on: {}".format(inp_date))
                    if(print_flag=='y' or print_flag=='Y'):
                        for center in resp_json["centers"]:
                            for session in center["sessions"]:
                                if session["min_age_limit"] <= age :
                                    print(session["min_age_limit"])
#                                     print("\t", center["name"])
#                                     print("\t", center["block_name"])
#                                     print("\t Price: ", center["fee_type"])
#                                     print("\t Available Capacity: ", session["available_capacity"])
                                    if(session["vaccine"] != ''):
#                                         print("\t Vaccine: ", session["vaccine"])
                                        pass
                                        
                                    else:
                                        
                                        session["vaccine"]='No Information'
#                                     print("\n\n")
                                    available_df.append([center["name"],center["block_name"],center["fee_type"],session["vaccine"],\
                                                        center['district_name'],center['state_name'],\
                                                        
                                                        center['pincode'],inp_date, session["available_capacity"], session["min_age_limit"]
                                                        
                                                        ])
                                                        
                           
                                 

                        
                else:
                    print("No available slots on {}".format(inp_date))


                available_df[0]



        # Create the pandas DataFrame for re grouping whole data into one particlur data frames
        info_vaccination = pd.DataFrame(available_df, columns = ['Vaccination Center', 'Block_name','Fee_type','Vaccine Type',\
            'District','State Name','pincode','Available Date','available capacity','Age Group'],index=None)
        
        indexNames = info_vaccination[ (info_vaccination['available capacity'] == 0) ].index
    
        info_vaccination.drop(indexNames , inplace=True)
       
        html=info_vaccination#.set_index(['Available Date', 'State Name','District','pincode'])
        
        return html

    except Exception as e:
        print("Exception ",e)




2021-05-12 10:18:07.616234
[datetime.datetime(2021, 5, 12, 10, 18, 7, 616234), datetime.datetime(2021, 5, 13, 10, 18, 7, 616234), datetime.datetime(2021, 5, 14, 10, 18, 7, 616234), datetime.datetime(2021, 5, 15, 10, 18, 7, 616234), datetime.datetime(2021, 5, 16, 10, 18, 7, 616234), datetime.datetime(2021, 5, 17, 10, 18, 7, 616234), datetime.datetime(2021, 5, 18, 10, 18, 7, 616234), datetime.datetime(2021, 5, 19, 10, 18, 7, 616234), datetime.datetime(2021, 5, 20, 10, 18, 7, 616234), datetime.datetime(2021, 5, 21, 10, 18, 7, 616234), datetime.datetime(2021, 5, 22, 10, 18, 7, 616234), datetime.datetime(2021, 5, 23, 10, 18, 7, 616234), datetime.datetime(2021, 5, 24, 10, 18, 7, 616234), datetime.datetime(2021, 5, 25, 10, 18, 7, 616234), datetime.datetime(2021, 5, 26, 10, 18, 7, 616234), datetime.datetime(2021, 5, 27, 10, 18, 7, 616234), datetime.datetime(2021, 5, 28, 10, 18, 7, 616234), datetime.datetime(2021, 5, 29, 10, 18, 7, 616234), datetime.datetime(2021, 5, 30, 10, 18, 7, 616234), dat

In [3]:
val=vaccineSlotsByDist(age=55,district_name='New Delhi',state='Delhi')


761
     State_id                   State_name                    district_name  \
0           1  Andaman and Nicobar Islands                          Nicobar   
1           1  Andaman and Nicobar Islands         North and Middle Andaman   
2           1  Andaman and Nicobar Islands                    South Andaman   
15          2               Andhra Pradesh  YSR District, Kadapa (Cuddapah)   
13          2               Andhra Pradesh                     Vizianagaram   
..        ...                          ...                              ...   
759        39                         None                            Mandi   
756        39                         None                           Chamba   
757        39                         None                           Kangra   
758        39                         None                          Kinnaur   
760        39                         None                           Shimla   

     district_id  
0              3  
1       

In [4]:
# ','.join(val['Vaccination Center'].unique().tolist()[:10])

val['District'].unique().tolist()[0]

# val['State Name'].unique().tolist()[0]


'New Delhi'

In [5]:
'''Twitter Updatation'''

import tweepy
import webbrowser
import time
from tweepy import OAuthHandler

import tweepy
from tweepy import StreamListener
from tweepy import Stream
import webbrowser


def tweet_data(state,dist,place):
##tweet

    CONSUMER_KEY='JTZluB3daESewe2ToGofBEdFG'
    CONSUMER_SECRET='hg3BGax9d5TiHw4UmB117WQYKE6Gnhe1dOL0zYyb4DfFlkH5HJ'
    Access_Token='1389917390520033284-of3M07i7EFEtwaM3GtoYkd62vSvaaA'
    Access_Token_Secret='z3ECIPDP6Lh7vbe0k1v0nYxer6GsvCnumsQ3Xtlsr7MqH'


    # callback_uri='oob'
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(Access_Token,Access_Token_Secret)
    api=tweepy.API(auth)
    
    print(state,dist,place)
    try:
        api.update_status("#{} Vaccine slots avaialble: \n {}.\n Visit VaxTrack for more info on Slots!".format(dist,place))
        print("TWeet Successful: #{} Vaccine slots: are \n {}".format(dist,place))
    except Exception as e:
        print("Exception in Tweeter API status Update: ",e)
    #val.destroy()
    

    
place=','.join(val['Vaccination Center'].unique().tolist()[:10])

dist='_'.join(val['District'].unique().tolist()[0].split(' '))

state=val['State Name'].unique().tolist()[0]
tweet_data(state,dist,place)
    


Delhi New_Delhi LHMC CVC 2,Charak Palika Hospial Site 1,Charak Palika Hospital Site 2,MCW Sarojini Nagar NDMC PHC,MCW Palika Health Complex,DGD Budh Nagar PHC,RML Site 3,RML Site 1,RML Site 2,Northern Railway Central
TWeet Successful: #New_Delhi Vaccine slots: are 
 LHMC CVC 2,Charak Palika Hospial Site 1,Charak Palika Hospital Site 2,MCW Sarojini Nagar NDMC PHC,MCW Palika Health Complex,DGD Budh Nagar PHC,RML Site 3,RML Site 1,RML Site 2,Northern Railway Central


In [6]:
gspread=gspreadData()

for i,j in zip(gspread['City'],gspread['State']):
    print(i,j)
    

           City        State
0     Hyderabad    Telengana
1  Bhubaneshwar       Odisha
2       Kolkata  West Bengal
3        bhbdha          NaN
Hyderabad Telengana
Bhubaneshwar Odisha
Kolkata West Bengal
bhbdha nan


In [7]:
gspread

,City,State
0,Hyderabad,Telengana
1,Bhubaneshwar,Odisha
2,Kolkata,West Bengal
3,bhbdha,NaN


In [8]:
for column in gspread[['City', 'State']]:
     
    # Select column contents by column   
    # name using [] operator
    print(column)
   

City
State


In [9]:
from multiprocessing import Value                    #This is Multithreading lib

counter = Value('i', 0)

def id_generator():
    '''
    This function that will generate id’s for each document.
    '''
    with counter.get_lock():
        counter.value += 1
        return counter.value
    
id_generator()

1

In [10]:
'_'.join(val['District'].unique().tolist()[0].split(' '))

'New_Delhi'

In [11]:
se='BBSR'

In [12]:
'_'.join(se.split(' '))

'BBSR'